In [3]:
import pandas as pd
import sqlite3

In [4]:
# Connect to SQLite database
conn = sqlite3.connect("x_data.db")
cursor = conn.cursor()

# 🔥 Drop the old table (only if you want to reset!)
cursor.execute("DROP TABLE IF EXISTS twitter_posts")

# ✅ Create table with correct schema
cursor.execute("""
CREATE TABLE twitter_posts (
    id TEXT PRIMARY KEY,
    author_id TEXT,
    username TEXT,
    text TEXT,
    created_at TEXT,
    like_count INTEGER,
    retweet_count INTEGER,
    reply_count INTEGER,
    quote_count INTEGER,
    media_url TEXT,
    source TEXT,
    language TEXT,
    api_completed_timestamp TEXT
)
""")

# Commit and close the database connection
conn.commit()
conn.close()

In [43]:
import requests
import sqlite3
import datetime

# Set up Bearer Token (Replace with your actual token)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPjpzwEAAAAAs%2FQnzL%2FMiOmEjyOXk9ElY0n1tw8%3D3pAhfbNjswssgDM69AxRjdUiX6VHcqiJHO0Nh67xYleSneiek0"

# Define the hashtag you want to search for (without #)
HASHTAG = "macavai"

# Twitter API endpoint for recent tweets with a hashtag
url = "https://api.twitter.com/2/tweets/search/recent"

# Define query parameters
params = {
    "query": f"#{HASHTAG} -is:reply",  # Exclude replies
    "tweet.fields": "id,text,created_at,public_metrics,attachments,lang,source,author_id,entities",
    "expansions": "attachments.media_keys,author_id",
    "media.fields": "media_key,type,url",
    "user.fields": "id,username",
    "max_results": 10,  # Adjust as needed (max 100)
}

# Set up headers with Bearer Token
headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}"
}

# Make API request
response = requests.get(url, params=params, headers=headers)
tweets_data = response.json()
print(tweets_data)

{'data': [{'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 1, 'quote_count': 0, 'bookmark_count': 0, 'impression_count': 3}, 'entities': {'hashtags': [{'start': 29, 'end': 35, 'tag': 'macav'}, {'start': 36, 'end': 44, 'tag': 'macavai'}], 'annotations': [{'start': 30, 'end': 34, 'probability': 0.6933, 'type': 'Other', 'normalized_text': 'macav'}, {'start': 37, 'end': 43, 'probability': 0.6652, 'type': 'Other', 'normalized_text': 'macavai'}]}, 'author_id': '1901356168121028608', 'created_at': '2025-03-19T12:55:05.000Z', 'lang': 'en', 'id': '1902343066830172663', 'text': 'Extremely happy right now :) #macav #macavai', 'edit_history_tweet_ids': ['1902343066830172663']}], 'includes': {'users': [{'id': '1901356168121028608', 'name': 'Macav.AI', 'username': 'Macav_ai'}]}, 'meta': {'newest_id': '1902343066830172663', 'oldest_id': '1902343066830172663', 'result_count': 1}}


In [44]:
display(tweets_data)

{'data': [{'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 1,
    'quote_count': 0,
    'bookmark_count': 0,
    'impression_count': 3},
   'entities': {'hashtags': [{'start': 29, 'end': 35, 'tag': 'macav'},
     {'start': 36, 'end': 44, 'tag': 'macavai'}],
    'annotations': [{'start': 30,
      'end': 34,
      'probability': 0.6933,
      'type': 'Other',
      'normalized_text': 'macav'},
     {'start': 37,
      'end': 43,
      'probability': 0.6652,
      'type': 'Other',
      'normalized_text': 'macavai'}]},
   'author_id': '1901356168121028608',
   'created_at': '2025-03-19T12:55:05.000Z',
   'lang': 'en',
   'id': '1902343066830172663',
   'text': 'Extremely happy right now :) #macav #macavai',
   'edit_history_tweet_ids': ['1902343066830172663']}],
 'includes': {'users': [{'id': '1901356168121028608',
    'name': 'Macav.AI',
    'username': 'Macav_ai'}]},
 'meta': {'newest_id': '1902343066830172663',
  'oldest_id': '1902343066830172663',
  're

In [51]:
tweets_data['data'][0]['entities']['hashtags'][1]['tag']

'macavai'

In [59]:
# Extract data
tweet = tweets_data['data'][0]
user = tweets_data['includes']['users'][0]

tweet_id = tweet['id']
text = tweet['text']
hashtags = ",".join([tag['tag'] for tag in tweet.get('entities', {}).get('hashtags', [])])
author_id = tweet['author_id']
username = user['username']
created_at = tweet['created_at']

print(hashtags)

macav,macavai


In [66]:
import sqlite3
from datetime import datetime

# Sample data from API

# Extract data
tweet = tweets_data['data'][0]
user = tweets_data['includes']['users'][0]

tweet_id = tweet['id']
text = tweet['text']
hashtags = ",".join([tag['tag'] for tag in tweet.get('entities', {}).get('hashtags', [])])
author_id = tweet['author_id']
geo = tweet.get("geo", {}).get("place_id", "N/A")
username = user['username']
created_at = tweet['created_at']

# Public metrics
metrics = tweet['public_metrics']
retweet_count = metrics['retweet_count']
reply_count = metrics['reply_count']
like_count = metrics['like_count']
quote_count = metrics['quote_count']
bookmark_count = metrics['bookmark_count']
impression_count = metrics['impression_count']

# Get API completed timestamp (current time in UTC)
api_completed_timestamp = datetime.utcnow().isoformat()

# Database connection
conn = sqlite3.connect("x.db")
cursor = conn.cursor()

# Create table with API completed timestamp
cursor.execute("""
    CREATE TABLE IF NOT EXISTS x_posts (
    id TEXT PRIMARY KEY,
    text TEXT,
    hashtags TEXT,
    author_id TEXT,
    username TEXT,
    created_at TEXT,
    retweet_count INTEGER,
    reply_count INTEGER,
    like_count INTEGER,
    quote_count INTEGER,
    bookmark_count INTEGER,
    impression_count INTEGER,
    geo_place_id TEXT, 
    api_completed_timestamp TEXT
    )
""")

# Insert data
cursor.execute("""
    INSERT OR REPLACE INTO x_posts (id, text, hashtags, author_id, username, created_at,
                                   retweet_count, reply_count, like_count, quote_count,
                                   bookmark_count, impression_count, geo_place_id, api_completed_timestamp)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (tweet_id, text, hashtags, author_id, username, created_at,
      retweet_count, reply_count, like_count, quote_count, bookmark_count,
      impression_count, geo, api_completed_timestamp))

# Commit and close connection
conn.commit()
conn.close()

print("Data inserted successfully with API timestamp:", api_completed_timestamp)

Data inserted successfully with API timestamp: 2025-03-19T22:19:25.963911


In [1]:
import sqlite3
import pandas as pd

# Connect to SQLite database
db_name = "x.db"
conn = sqlite3.connect(db_name)

# Read data into a Pandas DataFrame
query = "SELECT * FROM x_posts"
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Display the first few rows
display(df)

,id,text,hashtags,author_id,username,created_at,retweet_count,reply_count,like_count,quote_count,bookmark_count,impression_count,geo_place_id,api_completed_timestamp
0,1902343066830172663,Extremely happy right now :) #macav #macavai,"macav,macavai",1901356168121028608,Macav_ai,2025-03-19T12:55:05.000Z,0,0,1,0,0,3,N/A,2025-03-19T22:19:25.963911


In [10]:
rate_limit_url = "https://api.twitter.com/2/tweets/search/recent"
rate_limit_headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}"
}
response = requests.get(rate_limit_url, headers=rate_limit_headers)
print(response.headers)

{'Date': 'Wed, 19 Mar 2025 20:50:26 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '192', 'Connection': 'keep-alive', 'perf': '7402827104', 'Set-Cookie': 'guest_id=v1%3A174241742671049508; Max-Age=34214400; Expires=Sun, 19 Apr 2026 20:50:26 GMT; Path=/; Domain=.twitter.com; Secure; SameSite=None, __cf_bm=m.kwK69dWiL0UTnjjvs4ImSE8YayI9LB1oycsAhtGvU-1742417426-1.0.1.1-nvhfx1j1fRt0pobzcGCkCCzR7WEJAlkDdGiKkY80jKHJRwI_gSxo.a.65OpW.1_rbSBvLPxEcT8EMAm6iZSY0HBrnWYjr_SjtUwx0yIm8og; path=/; expires=Wed, 19-Mar-25 21:20:26 GMT; domain=.twitter.com; HttpOnly; Secure; SameSite=None', 'api-version': '2.131', 'Cache-Control': 'no-cache, no-store, max-age=0', 'x-access-level': 'read', 'x-frame-options': 'SAMEORIGIN', 'Content-Encoding': 'gzip', 'x-transaction-id': 'b3932e0d6ef1ba0e', 'x-xss-protection': '0', 'x-rate-limit-limit': '1', 'x-rate-limit-reset': '1742418326', 'content-disposition': 'attachment; filename=json.json', 'x-content-type-options': 'nosniff', 'x-rate-lim

In [11]:
import datetime

reset_time = datetime.datetime.utcfromtimestamp(1742390900)
print("Rate limit resets at:", reset_time)

Rate limit resets at: 2025-03-19 13:28:20


In [12]:
def check_rate_limit():
    url = "https://api.twitter.com/2/tweets/search/recent"
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    response = requests.get(url, headers=headers)
    
    print("Remaining Requests:", response.headers.get("x-rate-limit-remaining"))
    print("Rate Limit Resets At:", response.headers.get("x-rate-limit-reset"))  # UNIX timestamp

check_rate_limit()
reset_time = datetime.datetime.utcfromtimestamp(1742418326)
print(reset_time)

Remaining Requests: 0
Rate Limit Resets At: 1742418326
2025-03-19 21:05:26
